Imports

In [12]:
import cv2
import pytesseract

#comparar strings y encontrar coincidencias aproximadas:
import difflib 

import numpy as np

#crea ventanas gráficas simples:
from easygui import *

#expresiones regulares (limpiar texto OCR):
import re

#importa utilidades de estructuras de datos:
import collections
import requests
from urllib.request import Request, urlopen as uReq
from bs4 import BeautifulSoup as soup
import time
import unicodedata
import re
import json
import os
from rapidfuzz import process

Corregir un error de compatibilidad entre versiones antiguas y nuevas de Python donde Mapping y Sequence cambiaron de ubicación:

In [13]:
collections.Mapping = collections.abc.Mapping
collections.Sequence = collections.abc.Sequence

Configuración de Tesseract

In [14]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

Funcion para descargar la información de las cartas desde Scryfall API.

In [15]:

def normalize_text(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    text = re.sub(r'[^a-z0-9 ]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


def get_best_price(card):
    
    prices = card.get("prices", {})

    eur = prices.get("eur")
    eur_foil = prices.get("eur_foil")

    if eur:
        return float(eur)
    elif eur_foil:
        return float(eur_foil)
    else:
        return 0.0


def download_set_cards(set_code, output_folder="."):
    
    set_code = set_code.lower()
    url = f"https://api.scryfall.com/cards/search?q=e:{set_code}&unique=prints"

    cards_dict = {}

    while url:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        for card in data["data"]:

            name = card["name"]
            normalized_name = normalize_text(name)
            price = get_best_price(card)

            # Si no existe aún, guardar
            if normalized_name not in cards_dict:
                cards_dict[normalized_name] = {
                    "name": name,
                    "normalized_name": normalized_name,
                    "set": set_code.upper(),
                    "price_eur": price
                }

            else:
                # Si ya existe, quedarse con el precio más alto
                if price > cards_dict[normalized_name]["price_eur"]:
                    cards_dict[normalized_name]["price_eur"] = price

        url = data.get("next_page") if data.get("has_more") else None

    cards_list = list(cards_dict.values())

    filename = f"{set_code}_cards.json"
    filepath = os.path.join(output_folder, filename)

    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(cards_list, f, indent=4, ensure_ascii=False)

    print(f"Set optimizado guardado en: {filepath}")
    print(f"Total cartas únicas: {len(cards_list)}")

    return filepath



In [16]:
download_set_cards("ECL")

Set optimizado guardado en: .\ecl_cards.json
Total cartas únicas: 278


'.\\ecl_cards.json'

In [17]:
JSON_PATH = "ecl_cards.json"

with open(JSON_PATH, "r", encoding="utf-8") as f:
    cards = json.load(f)

cards_lookup = {card["normalized_name"]: card for card in cards}

In [19]:
def preprocess_for_ocr(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Aumentar contraste
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray = clahe.apply(gray)

    blur = cv2.GaussianBlur(gray, (3,3), 0)

    thresh = cv2.adaptiveThreshold(
        blur, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        11, 2
    )

    # Sharpen
    kernel = np.array([[0,-1,0],
                       [-1,5,-1],
                       [0,-1,0]])
    thresh = cv2.filter2D(thresh, -1, kernel)

    return thresh

In [25]:
def stackImages(scale, imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range(0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape[:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]),
                                                None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y] = cv2.cvtColor(imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank] * rows
        hor_con = [imageBlank] * rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None, scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor = np.hstack(imgArray)
        ver = hor
    return ver

In [26]:
def getPrice(cards_lookup, img, width, height):
    
    #card_title = img[25:75, 34:400]
    # cv2.imshow("Card Name",card_title)

    #--Nuevo--
    img = cv2.resize(img, (500, 500))
    h, w, _ = img.shape
    card_title = img[int(0.04*h):int(0.12*h), int(0.07*w):int(0.80*w)]
    #-------------
    

    #card_name = re.sub('[^a-zA-Z0-9,+ ]', '', pytesseract.image_to_string(card_title))
    #--Nuevo--
    card_name = preprocess_for_ocr(card_title)
    custom_config = r'--oem 3 --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789 '
    raw_text = pytesseract.image_to_string(card_title, config=custom_config)
    # 🔹 Normalizamos texto igual que Scryfall
    normalized_ocr = normalize_text(raw_text)
    #-----


    # print("Card name: ", card_name)
    #closest_match = difflib.get_close_matches(card_name, cards_lookup.keys())

    #--Nuevo--
    match = process.extractOne(normalized_ocr, cards_lookup.keys())

    if match and match[1] > 75:
        best_match = match[0]
        card_info = cards_lookup[best_match]

        raw_price = str(card_info.get("price_eur", "0.00"))
        price = re.sub('[^0-9$.]', '', raw_price)

        return card_info["name"], price
    else:
        return "Not Found in Chosen Set", ""

    """
    if len(closest_match) >= 1:
        # print("Closest Matches",closest_match)
        closest_match = closest_match[0]
        # OBTENEMOS LA INFO DE LA CARTA
        card_info = cards_lookup[closest_match]
        #price = re.sub('[^0-9$.]', '', cards_lookup[closest_match])
        #price = re.sub('[^0-9$.]', '', cards_lookup[closest_match]["price"])
        raw_price = str(card_info.get("price_eur", "0.00"))
        price = re.sub('[^0-9$.]', '', raw_price)
    else:
        closest_match = "Not Found in Chosen Set"
        price = ""

    return closest_match, price
    """

In [27]:
def getContours(img, imgContour, originalImg, imgConts, cards_lookup):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    width, height = 500, 700
    
    # Valores por defecto para evitar errores si no entra en el if
    card_name = ""
    price = ""

    for cnt in contours:
        area = cv2.contourArea(cnt)
        areaMin = 20000 
            
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
        
        if area > areaMin and len(approx) == 4:
            cv2.drawContours(imgConts, cnt, -1, (255, 0, 255), 7)
            rectX, rectY, rectW, rectH = cv2.boundingRect(approx)
            cv2.rectangle(imgContour, (rectX, rectY), (rectX + rectW, rectY + rectH), (0, 255, 0), 5)
            
            points = []
            for point in approx:
                x, y = point[0]
                points.append([x, y])
            
            # Tu lógica de warping (tal cual la tenías)
            card = np.float32(points)
            x1, y1 = points[0]
            x2, y2 = points[1]
            x3, y3 = points[2]
            x4, y4 = points[3]

            if np.sqrt(np.square(x1 - x2) + np.square(y1 - y2)) < np.sqrt(np.square(x1 - x4) + np.square(y1 - y4)):
                # top point goes to top right
                cardWarped = np.float32([[width, 0], [0, 0], [0, height], [width, height]])
            else:
                # top point goes to top left
                cardWarped = np.float32([[width, 0], [0, 0], [0,height], [width, height]])
            
            matrix = cv2.getPerspectiveTransform(card, cardWarped)
            imgOutput = cv2.warpPerspective(originalImg, matrix, (width, height))
            
            # --- AQUÍ ESTABA EL ERROR PRINCIPAL ---
            # Usamos cards_lookup (el argumento) en lugar de setMap
            card_name, price = getPrice(cards_lookup, imgOutput, width, height)
            
            if card_name == "Not Found":
                textColor = (0, 0, 255) # Rojo si falla
                display_text = "Desconocida"
            else:
                textColor = (0, 255, 0) # Verde si acierta
                display_text = f"{card_name} | {price}"

            # Ponemos el texto
            cv2.putText(imgContour, display_text, (rectX, rectY - 20), cv2.FONT_HERSHEY_COMPLEX, .7, textColor, 2)

In [31]:
frameWidth = 700
frameHeight = 700
cap = cv2.VideoCapture("prueba_magic.mp4")
cap.set(3, frameWidth)
cap.set(4, frameHeight)

while True:
    succes, img = cap.read()
    imgContour = img.copy()
    imgConts = img.copy()

    imgBlur = cv2.GaussianBlur(img, (5,5), 0)
    imgGray = cv2.cvtColor(imgBlur, cv2.COLOR_BGR2GRAY)

    """
    #threshold1 = cv2.getTrackbarPos("Threshold1", "Parameters")
    #threshold2 = cv2.getTrackbarPos("Threshold2", "Parameters")
    #threshold1 = 80
    #threshold2 = 200
    #imgCanny = cv2.Canny(imgGray, threshold1, threshold2)

    #v = np.median(imgGray)

    #sigma = 0.33
    #threshold1 = int(max(0, (1.0 - sigma) * v))
    #threshold2 = int(min(255, (1.0 + sigma) * v))
    """

    #imgCanny = cv2.Canny(imgGray, 50, 150)

    v = np.median(imgGray)
    sigma = 0.33
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    imgCanny = cv2.Canny(imgGray, lower, upper)


    kernel = np.ones((5, 5), np.uint8)
   
    imgDil = cv2.dilate(imgCanny, kernel, iterations=1)

    getContours(imgDil, imgContour, img, imgConts, cards_lookup)

    mgStack = stackImages(0.8, ([img, imgGray, imgCanny],
                                 [imgConts, imgContour, img]))
    
    cv2.namedWindow("MTG Price Lookup OpenCV Project", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("MTG Price Lookup OpenCV Project", 700, 700)
    cv2.imshow("MTG Price Lookup OpenCV Project", imgContour)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()